In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import re


In [ ]:
true=pd.read_csv("")
fake=pd.read_csv("")

In [ ]:
true['label']=np.ones(len(true),dtype=int)
fake['label']=np.zeros(len(fake),dtype=int)

In [ ]:
data=pd.concat((true,fake),axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [ ]:
data= data.sample(frac=1)


In [ ]:
data = data.drop("date",axis=1)

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
lemma = WordNetLemmatizer()
pattern = "[^a-zA-Z]"
def preprocess_text(text):
    text = re.sub(pattern, " ", text)  # Cleaning
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmaized_tokens  = [lemma.lemmatize(word) for word in filtered_tokens]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return stemmed_tokens

In [ ]:
all_tokens = [preprocess_text(text) for text in data['text']] + [preprocess_text(title) for title in data['title']]
model = Word2Vec(all_tokens, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def vectorize_text(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if not vectors:
        # If none of the words are in the model's vocabulary, return None
        return None
    # Average the word vectors to get the text representation
    avg_vector = sum(vectors) / len(vectors)
    return avg_vector

In [ ]:
# Vectorize the "text" column
data['text_vector'] = data['text'].apply(lambda x: vectorize_text(preprocess_text(x), model))

# Vectorize the "title" column
data['title_vector'] = data['title'].apply(lambda x: vectorize_text(preprocess_text(x), model))

# Print the new DataFrame
print(data)

                                                   title  \
17271  Iraqi forces seize oil city Kirkuk from Kurds ...   
4033    Democrats SUE Republicans Over Plot To Rig El...   
21474  HILLARY EXPOSED: WATCH UNCOVERED VIDEO The Hil...   
1366    Trump Might Be About To Viciously Sabotage He...   
1338    Rick Santorum Advises President To Quit Tweet...   
...                                                  ...   
4979   Trump gives CIA authority to conduct drone str...   
5428   Netanyahu asks Trump to recognize Israeli sove...   
7497   More first-time voters, late-deciders in U.S. ...   
16125  LOU DOBBS Destroys Paul Ryan In Epic Rant: “Ju...   
9660   Former Texas Gov. Perry endorses Trump, says o...   

                                                    text          subject  \
17271  BAGHDAD/KIRKUK, Iraq (Reuters) - Iraqi governm...        worldnews   
4033   Democrats appear to be completely fed up with ...             News   
21474  Hillary shows her true colors when a fema

In [ ]:
data = data.dropna()

In [ ]:
X_text = np.array(data['text_vector'].tolist())
X_title = np.array(data['title_vector'].tolist())
X_combined = np.hstack((X_text, X_title))  # Concatenate both vectors
y = np.array(data['label'])

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras.utils import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.layers import LeakyReLU

In [ ]:
maxlen = 100
X_train_pad = pad_sequences(x_train, maxlen=maxlen)
X_test_pad = pad_sequences(x_test, maxlen=maxlen)

# Convert the labels to categorical
num_classes = len(np.unique(y_train))
Y_train_cat = to_categorical(y_train, num_classes)
Y_test_cat = to_categorical(y_test, num_classes)

# Build the CNN model


# Define the model
model = Sequential()

model.add(Conv1D(32, kernel_size=2, input_shape=(maxlen, 1)))
leaky_relu = LeakyReLU(alpha=0.1) # alpha is the slope of the negative part
model.add(leaky_relu)
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(64))
model.add(leaky_relu)
model.add(Dropout(0.6))
model.add(Dense(num_classes, activation='sigmoid'))

from keras.optimizers import RMSprop

learning_rate = 0.001
optimizer = RMSprop(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
batch_size = 124
epochs = 10
model.fit(X_train_pad.reshape(-1, maxlen, 1), Y_train_cat, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad.reshape(-1, maxlen, 1), Y_test_cat))

Epoch 1/10
286/286 [==============================] - 8s 22ms/step - loss: 0.3060 - accuracy: 0.8755 - val_loss: 0.2823 - val_accuracy: 0.8858
Epoch 2/10
286/286 [==============================] - 9s 31ms/step - loss: 0.2807 - accuracy: 0.8861 - val_loss: 0.2788 - val_accuracy: 0.8883
Epoch 3/10
286/286 [==============================] - 5s 18ms/step - loss: 0.2760 - accuracy: 0.8890 - val_loss: 0.2765 - val_accuracy: 0.8890
Epoch 4/10
286/286 [==============================] - 5s 18ms/step - loss: 0.2727 - accuracy: 0.8915 - val_loss: 0.2731 - val_accuracy: 0.8909
Epoch 5/10
286/286 [==============================] - 7s 25ms/step - loss: 0.2720 - accuracy: 0.8916 - val_loss: 0.2723 - val_accuracy: 0.8918
Epoch 6/10
286/286 [==============================] - 6s 22ms/step - loss: 0.2689 - accuracy: 0.8936 - val_loss: 0.2739 - val_accuracy: 0.8895
Epoch 7/10
286/286 [==============================] - 8s 27ms/step - loss: 0.2669 - accuracy: 0.8932 - val_loss: 0.2708 - val_accuracy: 0.8918

In [ ]:
from sklearn.metrics import confusion_matrix
scores = model.evaluate(X_test_pad.reshape(-1, maxlen, 1), Y_test_cat)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Get the predicted labels for the test set
Y_pred = model.predict(X_test_pad.reshape(-1, maxlen, 1))
Y_pred = np.argmax(Y_pred, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_test, Y_pred)
print('Confusion matrix:')
print(cm)

277/277 [==============================] - 1s 5ms/step - loss: 0.2732 - accuracy: 0.8901
Test loss: 0.27323073148727417
Test accuracy: 0.8901061415672302
277/277 [==============================] - 1s 4ms/step
Confusion matrix:
[[4227  334]
 [ 639 3654]]
